In [1]:
import json
import requests
import mercury as mr
import geemap
import ollama

Map = geemap.Map(ee_initialize=False, center=[52.23, 21.01], zoom=12)
Map.add_basemap("OpenStreetMap")


In [3]:
# Shared state: AOI bbox in Overpass order: [south, west, north, east]
STATE = {"aoi_bbox": None}

def _overpass_to_geojson(data: dict) -> dict:
    """Convert Overpass JSON to GeoJSON FeatureCollection (nodes + ways)."""
    elements = data.get("elements", [])
    feats = []

    for el in elements:
        etype = el.get("type")
        tags = el.get("tags") or {}
        name = tags.get("name") or tags.get("brand") or ""

        if etype == "node":
            lat, lon = el.get("lat"), el.get("lon")
            if lat is None or lon is None:
                continue
            feats.append({
                "type": "Feature",
                "properties": {"name": name, **tags},
                "geometry": {"type": "Point", "coordinates": [lon, lat]},
            })

        elif etype == "way":
            geom = el.get("geometry")
            if not geom:
                continue
            coords = [[p["lon"], p["lat"]] for p in geom if "lon" in p and "lat" in p]
            if len(coords) < 2:
                continue

            # closed -> polygon, else -> line
            if coords[0] == coords[-1] and len(coords) >= 4:
                geometry = {"type": "Polygon", "coordinates": [coords]}
            else:
                geometry = {"type": "LineString", "coordinates": coords}

            feats.append({
                "type": "Feature",
                "properties": {"name": name, **tags},
                "geometry": geometry,
            })

    return {"type": "FeatureCollection", "features": feats}


In [4]:
def set_view(lat: float, lon: float, zoom: int = 12) -> str:
    """Set map center and zoom."""
    Map.set_center(lon, lat, zoom)
    return f"✅ View set to lat={lat}, lon={lon}, zoom={zoom}"

def set_basemap(name: str = "OpenStreetMap") -> str:
    """Set basemap. Examples: OpenStreetMap, Esri.WorldImagery, OpenTopoMap, CartoDB.DarkMatter."""
    Map.add_basemap(name)
    return f"✅ Basemap set to {name}"

def clear_layers() -> str:
    """Remove all added layers (keeps basemap)."""
    for layer in list(Map.layers)[1:]:
        Map.remove_layer(layer)
    return "✅ Cleared layers"

def add_marker(lat: float, lon: float, label: str = "📍") -> str:
    """Add a marker with tooltip label."""
    Map.add_marker(location=(lat, lon), tooltip=label)
    return f"✅ Marker added: {label} at ({lat}, {lon})"

def set_aoi(bbox: list) -> str:
    """Set AOI bbox: [south, west, north, east] (Overpass order)."""
    if not (isinstance(bbox, list) and len(bbox) == 4):
        raise ValueError("bbox must be [south, west, north, east]")

    STATE["aoi_bbox"] = bbox
    south, west, north, east = bbox

    rect_geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {"name": "AOI"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [west, south], [east, south], [east, north], [west, north], [west, south]
                ]]
            }
        }]
    }
    Map.add_geojson(rect_geojson, layer_name="AOI")
    Map.set_center((west + east) / 2, (south + north) / 2, 12)
    return f"✅ AOI set to {bbox}"

def osm_search(tag: str, limit: int = 200, layer_name: str | None = None) -> str:
    """
    Search OSM using Overpass within the current AOI.
    tag must be 'key=value' e.g. amenity=cafe
    """
    bbox = STATE.get("aoi_bbox")
    if not bbox:
        raise ValueError("AOI not set. Call set_aoi(bbox) first.")

    if "=" not in tag:
        raise ValueError("tag must be key=value (e.g. amenity=cafe)")

    k, v = [x.strip() for x in tag.split("=", 1)]
    south, west, north, east = bbox

    q = f"""
    [out:json][timeout:25];
    (
      node["{k}"="{v}"]({south},{west},{north},{east});
      way["{k}"="{v}"]({south},{west},{north},{east});
    );
    out geom {int(limit)};
    """

    r = requests.post(
        "https://overpass-api.de/api/interpreter",
        data=q.encode("utf-8"),
        headers={"Content-Type": "text/plain"},
        timeout=60,
    )
    r.raise_for_status()

    gj = _overpass_to_geojson(r.json())
    n = len(gj["features"])
    lname = layer_name or f"OSM {tag} ({n})"
    Map.add_geojson(gj, layer_name=lname)

    return f"✅ Added layer '{lname}' with {n} features"

def geocode_city(city: str) -> str:
    """Geocode a city name using OpenStreetMap Nominatim and set AOI + view."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": city,
        "format": "json",
        "limit": 1,
        "addressdetails": 0,
    }

    headers = {"User-Agent": "Mercury-GeoChat-Demo"}

    r = requests.get(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    data = r.json()

    if not data:
        raise ValueError(f"City not found: {city}")

    result = data[0]

    lat = float(result["lat"])
    lon = float(result["lon"])
    south, north, west, east = map(float, result["boundingbox"])
    bbox = [south, west, north, east]

    # Set AOI and view
    set_aoi(bbox)
    set_view(lat, lon, zoom=12)

    return f"✅ City '{city}' located. AOI and view updated."


TOOLS = [set_view, set_basemap, clear_layers, add_marker, set_aoi, osm_search, geocode_city]


In [5]:
SYSTEM_PROMPT = """
You are a geospatial map assistant inside a web app.
You can control the map ONLY by calling tools.

Rules:
- Prefer tool calls over explanations.
- If the user asks to "show/find/search" something and AOI is missing, ask ONE short question:
  "What area should I search? Provide bbox [south, west, north, east]."
- Use Overpass via osm_search(tag="key=value").

Supported tools:
- set_view(lat, lon, zoom)
- set_basemap(name)
- clear_layers()
- add_marker(lat, lon, label)
- set_aoi(bbox) where bbox=[south, west, north, east]
- osm_search(tag, limit=..., layer_name=...)
- geocode_city(city)

If the user mentions a city or place name, first call geocode_city.

Tag cheatsheet:
- schools -> amenity=school
- cafes -> amenity=cafe
- restaurants -> amenity=restaurant
- parking -> amenity=parking
- hospitals -> amenity=hospital
- pharmacies -> amenity=pharmacy
- playgrounds -> leisure=playground
- parks -> leisure=park
- museums -> tourism=museum
- hotels -> tourism=hotel
- supermarkets -> shop=supermarket

After tool use, reply with ONE short sentence confirming what you displayed.
"""


In [6]:
left, right = mr.Columns(2)

ColumnsBox(children=(ColumnOutput(layout=Layout(flex='1 1 0px', min_width='100px'), _dom_classes=('mljar-colum…

In [7]:
with right:
    display(Map)

In [8]:
with left:
    chat = mr.Chat()

In [9]:

messages = [{"role": "system", "content": SYSTEM_PROMPT}]


In [10]:
prompt = mr.ChatInput()

In [11]:

if prompt.value:
    # user message
    chat.add(mr.Message(markdown=prompt.value, role="user"))
    messages.append({"role": "user", "content": prompt.value})

    # call local LLM (Ollama)
    stream = ollama.chat(
        model="gpt-oss:120b",
        messages=messages,
        tools=TOOLS,
        stream=True,
    )

    ai_msg = mr.Message(role="assistant", emoji="🤖")
    chat.add(ai_msg)

    thinking, content = "", ""
    tool_calls = []

    for chunk in stream:
        if getattr(chunk.message, "thinking", None):
            if thinking == "":
                ai_msg.append_markdown("Thinking: ")
            thinking += chunk.message.thinking
            ai_msg.append_markdown(chunk.message.thinking)
            
            

        if getattr(chunk.message, "content", None):
            if content == "":
                ai_msg.append_markdown("\n\nAnswer: ")
            content += chunk.message.content
            ai_msg.append_markdown(chunk.message.content)

        if getattr(chunk.message, "tool_calls", None):
            tool_calls.extend(chunk.message.tool_calls)

    messages.append({
        "role": "assistant",
        "thinking": thinking,
        "content": content,
        "tool_calls": tool_calls,
    })

    # execute tools
    for call in tool_calls:
        name = call.function.name
        args = call.function.arguments  

        tool_msg = mr.Message(role="tool", emoji="🧰")
        chat.add(tool_msg)

        try:
            if name == "set_view":
                result = set_view(**args)
            elif name == "set_basemap":
                result = set_basemap(**args)
            elif name == "clear_layers":
                result = clear_layers()
            elif name == "add_marker":
                result = add_marker(**args)
            elif name == "set_aoi":
                result = set_aoi(**args)
            elif name == "osm_search":
                result = osm_search(**args)
            elif name == "geocode_city":
                result = geocode_city(**args)
            else:
                result = f"Unknown tool: {name}"

            tool_msg.append_markdown(result)

        except Exception as e:
            err = f"❌ Tool error in {name}: {e}"
            tool_msg.append_markdown(err)
            result = err

        # feed tool result back to the model
        messages.append({
            "role": "tool",
            "tool_name": name,
            "content": result
        })